In [ ]:
%cd ..

In [ ]:
import pandas as pd
import os
import time
from scipy.io.wavfile import read

In [ ]:
from IPython.display import Audio, clear_output
from src.common_paths import get_test_data_path

In [37]:
df = pd.read_csv("data/test/submission0.3.csv")

df = df.sample(frac=1, random_state=655321)
df_results = pd.read_csv(os.path.join(get_test_data_path(), "results0.3.csv"))
i = df_results.shape[0]-1
fnames = df_results.fname.tolist()
labels = df_results.label.tolist()
human_labels = df_results.human_label.tolist()
prev_label=None

In [21]:
%matplotlib nbagg

In [7]:
while 1:
    i += 1
    fname = df.iloc[i]["fname"]
    label = df.iloc[i]["label"]
    filepath = os.path.join(get_test_data_path(), "audio", fname)
    sr, wav = read(filepath)
    clear_output(True)
    print("Previous label was:", prev_label)
    display(Audio(data=wav, rate=sr, autoplay=True))
    time.sleep(1.2)
    human_label = input()
    fnames.append(fname)
    labels.append(label)
    human_labels.append(human_label)
    df_results = pd.DataFrame({"fname": fnames, "label": labels, "human_label": human_labels})
    df_results.to_csv(os.path.join(get_test_data_path(), "results0.3.csv"))
    prev_label = label

Previous label was: down


one


KeyboardInterrupt: 

In [38]:
known_classes = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "unknown",
                              "silence"]

In [39]:
df_results["human_label_corrected"] = df_results["human_label"]

In [40]:
df_results.loc[~df_results.human_label_corrected.isin(known_classes), "human_label_corrected"] = "unknown"

In [45]:
df_results["correct"]=(df_results.human_label_corrected == df_results.label)

In [52]:
(df_results
 .assign(c=1)
 .groupby(["label", "human_label_corrected"])["c"]
 .sum()
 .reset_index()
 .pivot_table(values="c", index="label", columns="human_label_corrected")
 .fillna(0))

human_label_corrected,down,go,left,no,off,on,right,silence,stop,unknown,up,yes
label,,,,,,,,,,,,
down,13.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
go,3.0,14.0,0.0,1.0,0.0,1.0,0.0,3.0,1.0,3.0,1.0,0.0
left,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
no,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
off,0.0,1.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0
on,0.0,0.0,0.0,0.0,0.0,12.0,0.0,4.0,0.0,4.0,4.0,0.0
right,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,3.0,0.0,0.0
silence,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,0.0,2.0,0.0,1.0
stop,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,2.0,1.0,0.0


In [53]:
from sklearn.metrics import classification_report

In [55]:
print(classification_report(df_results.label, df_results.human_label_corrected))

             precision    recall  f1-score   support

       down       0.81      0.72      0.76        18
         go       0.88      0.52      0.65        27
       left       0.73      0.92      0.81        12
         no       0.74      0.82      0.78        17
        off       0.86      0.71      0.77        17
         on       0.80      0.50      0.62        24
      right       1.00      0.82      0.90        17
    silence       0.68      0.93      0.78        41
       stop       0.85      0.88      0.87        26
    unknown       0.92      0.92      0.92       286
         up       0.61      0.77      0.68        22
        yes       0.93      0.93      0.93        15

avg / total       0.86      0.85      0.85       522



In [58]:
df_results.to_excel("C:/Users/Ivan Valles Perez/Desktop/analysis.xlsx")